An example of the Code used for verifying the uniqueness check.

In [3]:

from sudoku_full_board import SudokuGenerator
from Solver_experiment_unified import UnifiedSolver
import random, copy

Generator1 = SudokuGenerator(9)
Generator1.generate_full_board()
print(Generator1.board)

#In a loop generate a random board, remove every instance of 1 and 2 and check if it has a unique solution, repeat 100 times 
correct = 0
for i in range(1000):
    Generator1 = SudokuGenerator(9)
    Generator1.generate_full_board()
    Board_1 = Generator1.board
    
    #Remove 1 and 2 from the board, ensurong it will have exatly two solutions, making it the hardest case to find if there are multiple solutions
    for i in range(9):
        for j in range(9):
            if Board_1[i][j] == 1 or Board_1[i][j] == 2:
                Board_1[i][j] = 0
    
    solver = UnifiedSolver(Board_1)
    if not solver.has_single_solution():
        correct += 1

print(correct/1000)

[[6, 4, 7, 1, 2, 3, 5, 8, 9], [8, 1, 3, 5, 9, 4, 2, 7, 6], [5, 2, 9, 8, 6, 7, 1, 4, 3], [1, 6, 8, 3, 4, 5, 9, 2, 7], [3, 7, 4, 2, 8, 9, 6, 5, 1], [2, 9, 5, 7, 1, 6, 4, 3, 8], [4, 8, 1, 6, 7, 2, 3, 9, 5], [7, 3, 2, 9, 5, 1, 8, 6, 4], [9, 5, 6, 4, 3, 8, 7, 1, 2]]
1.0


In [4]:
import pandas as pd
#read the csv file, ignore /n 

read_file = pd.read_csv('sudoku_results.csv')
read_file = read_file.replace('\n', '', regex=True)
read_file = read_file.replace(' ', '', regex=True)

#loop over the sudoku_generated column and check if the board has a unique solution

#create a new column to store the results
read_file['has_unique_solution'] = 0

for i in range(len(read_file)):
    board = read_file['sudoku_generated'][i]
    board = [[int(cell) if cell != '.' else 0 for cell in board[i:i+9]] for i in range(0, 81, 9)]
    solver = UnifiedSolver(board)
    read_file['has_unique_solution'][i] = solver.has_single_solution()

#print the proportion of boards that have a unique solution
print(read_file['has_unique_solution'])


C:\Users\killi\AppData\Local\Temp\ipykernel_24332\1224475321.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  read_file['has_unique_solution'][i] = solver.has_single_solution()
C:\Users\killi\AppData\Local\Temp\ipykernel_24332\1224475321.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  read_file['has_unique_solution'][i] = solver.has_single_solution()


0       True
1       True
2      False
3      False
4       True
       ...  
295    False
296    False
297     True
298    False
299    False
Name: has_unique_solution, Length: 300, dtype: object


This code is used for generating and storing unique sudokus:

In [27]:
import csv
from time import time, sleep
import pandas as pd
from sudoku_full_board import SudokuGenerator
from Solver_experiment_unified import UnifiedSolver
import random, copy
from sudoku_game_v5 import SudokuGame

def test_sudoku_solving(num_games=20):
    difficulties = ['easy', 'medium', 'hard']
    sudoku_data = []

    # Loop through each difficulty level
    for difficulty in difficulties:
        print(f"\nTesting difficulty: {difficulty.upper()}")
        
        # Generate and solve num_games Sudoku games per difficulty
        for _ in range(num_games):
            retry_attempts = 0
            puzzle_generated = False
            
            while not puzzle_generated and retry_attempts < 5:  # Retry up to 5 times if necessary
                try:
                    game = SudokuGame(difficulty=difficulty)
                    puzzle = game.generate_game()  # Generate the game board
                    puzzle_generated = True
                except Exception as e:
                    retry_attempts += 1
                    print(f"Attempt {retry_attempts}: Failed to generate puzzle, retrying...")
                    sleep(1)  # Wait for 1 second before retrying

            if not puzzle_generated:
                print("Failed to generate Sudoku puzzle after several attempts.")
                continue

            print("Generated Sudoku Puzzle:")
            generated_puzzle_str = "\n".join(" ".join(str(val) if val != 0 else "." for val in row) for row in puzzle)
            print(generated_puzzle_str)
            
            solver = UnifiedSolver(puzzle)
            if solver.solve():
                print("Sudoku Solved! Solution:")
                solved_puzzle_str = "\n".join(" ".join(str(val) for val in row) for row in solver.board)
                print(solved_puzzle_str)
                if solver.has_single_solution():
                    sudoku_data.append([generated_puzzle_str, solved_puzzle_str, difficulty])
                    print("Unique solution found.")
                else:
                    print("Multiple solutions found.")
            else:
                solved_puzzle_str = "Failed to solve the Sudoku."
                print(solved_puzzle_str)
    
    # Write results to a CSV file
    with open('sudoku_results.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['sudoku_generated', 'sudoku_solved', 'difficulty_level'])
        writer.writerows(sudoku_data)

    return sudoku_data


# Run the test
test_sudoku_solving(20)



Testing difficulty: EASY
Generated Sudoku Puzzle:
. 8 6 1 . . 7 . .
. . . 6 2 8 . 4 3
4 3 . . 5 . 2 6 .
5 . . . . . . 7 .
1 6 9 . 7 . 3 8 2
. 7 . . . . . . 4
. 1 3 . 8 . . 5 9
7 2 . 4 9 6 . . .
. . 4 . . 5 6 2 .
Sudoku Solved! Solution:
2 8 6 1 4 3 7 9 5
9 5 7 6 2 8 1 4 3
4 3 1 9 5 7 2 6 8
5 4 2 8 3 1 9 7 6
1 6 9 5 7 4 3 8 2
3 7 8 2 6 9 5 1 4
6 1 3 7 8 2 4 5 9
7 2 5 4 9 6 8 3 1
8 9 4 3 1 5 6 2 7
Unique solution found.
Generated Sudoku Puzzle:
8 1 2 . . . 6 . 7
. . 5 . 6 . 3 8 .
9 3 6 . . . . . 5
. 2 3 7 . . . . .
5 . . 3 2 4 . . 1
. . . . . 1 2 3 .
6 . . . . . 1 2 3
. 5 1 . 8 . 7 . .
3 . 7 . . . 5 6 8
Sudoku Solved! Solution:
8 1 2 4 3 5 6 9 7
4 7 5 1 6 9 3 8 2
9 3 6 2 7 8 4 1 5
1 2 3 7 9 6 8 5 4
5 6 8 3 2 4 9 7 1
7 9 4 8 5 1 2 3 6
6 8 9 5 4 7 1 2 3
2 5 1 6 8 3 7 4 9
3 4 7 9 1 2 5 6 8
Unique solution found.
Generated Sudoku Puzzle:
7 . . . 2 6 . 8 9
. . . 4 7 . 1 2 .
. 2 . 5 . 9 4 . .
. . . . . . 7 . 5
5 . 1 2 . 7 6 . 3
4 . 9 . . . . . .
. . 5 6 . 1 . 7 .
. 1 7 . 5 2 . . .
6 9 . 7 3 .

KeyboardInterrupt: 